In [8]:
from typing import Optional, Type
from langchain.tools.base import BaseTool
from typing import List
from pydantic import BaseModel, Field, ConfigDict
import json
from square.client import Client
from square.api.orders_api import OrdersApi

In [9]:
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [10]:
# put these in a file and read to environment variables...
CAN_ACCESS="EAAAEJ4obHl4oPUnC94AJosAYUWqiCOOPtcmDejyvE3Dd51F-zcC6PHvFq-HSwI4"
US_ACCESS="EAAAEP0FUqPfb4_jvHrDm3HVfDUcUpK6TQPyf2f-9S2nE-R17YBu0Qt-sN3DO9GU"

US_LOCATION="LETE3KSW0H1RB"

DEVICE_ID_CHECKOUT_SUCCESS="9fa747a2-25ff-48ee-b078-04381f7c828f"
DEVICE_ID_CHECKOUT_SUCCESS_TIP="22cd266c-6246-4c06-9983-67f0c26346b0"
DEVICE_ID_CHECKOUT_SUCCESS_GC="4mp4e78c-88ed-4d55-a269-8008dfe14e9"
DEVICE_ID_CHECKOUT_FAILURE_BUYERCANCEL="841100b9-ee60-4537-9bcf-e30b2ba5e215"

client = Client(
    square_version='2023-08-16',
    access_token=US_ACCESS,
    environment='sandbox')

In [26]:
# some helper functions for easy api debugging
def handle_response(result):
    if result.is_success():
      print(result.body)
    elif result.is_error():
      print(result.errors)

def search_orders(location: str, return_entries: bool =False):
    body = { 'location_ids': [location], "return_entries": return_entries}
    result = client.orders.search_orders(body)
    handle_response(result)


def create_terminal_checkout(order_id: str):
    result = client.terminal.create_terminal_checkout(
      body = {
        "idempotency_key": str(uuid4().hex),
        "checkout": {
          "amount_money": {
            "amount": 500,
            "currency": "USD"
          },
          "order_id": order_id,
          "payment_options": {
            "autocomplete": True,
            "accept_partial_authorization": False
          },
          "device_options": {
            "device_id": "123",
            "skip_receipt_screen": True,
            "collect_signature": False,
            "show_itemized_cart": False
          },
          "payment_type": "CARD_PRESENT",
          "customer_id": ""
        }
      }
    )
    handle_response(result)

def get_terminal_checkout(checkout_id: str):
    result = client.terminal.get_terminal_checkout(checkout_id)
    handle_response(result)


In [16]:
from google.cloud import aiplatform

aiplatform.init(project="loyal-karma-397523", location="us-central1")

# if authentication doesn't work with just above, uncomment below.
# !gcloud auth application-default set-quota-project loyal-karma-397523

In [21]:
create_terminal_checkout("iHLPW9xNW1h4SG8ubhqlRo5tg7aZY")

{'checkout': {'id': 'HPD522PpT2qqO', 'amount_money': {'amount': 500, 'currency': 'USD'}, 'device_options': {'device_id': '123', 'collect_signature': False, 'tip_settings': {'allow_tipping': False}, 'skip_receipt_screen': True, 'show_itemized_cart': False}, 'status': 'PENDING', 'created_at': '2023-08-31T15:59:17.450Z', 'updated_at': '2023-08-31T15:59:17.450Z', 'app_id': 'sandbox-sq0idb-eDfa0uKR4GvZ1SC5bMSGkw', 'order_id': 'iHLPW9xNW1h4SG8ubhqlRo5tg7aZY', 'location_id': 'LETE3KSW0H1RB', 'payment_type': 'CARD_PRESENT', 'customer_id': '', 'payment_options': {'autocomplete': True, 'accept_partial_authorization': False}}}


In [27]:
get_terminal_checkout("HPD522PpT2qqO")

{'checkout': {'id': 'HPD522PpT2qqO', 'amount_money': {'amount': 500, 'currency': 'USD'}, 'device_options': {'device_id': '123', 'collect_signature': False, 'tip_settings': {'allow_tipping': False}, 'skip_receipt_screen': True, 'show_itemized_cart': False}, 'status': 'PENDING', 'created_at': '2023-08-31T15:59:17.450Z', 'updated_at': '2023-08-31T15:59:17.450Z', 'app_id': 'sandbox-sq0idb-eDfa0uKR4GvZ1SC5bMSGkw', 'order_id': 'iHLPW9xNW1h4SG8ubhqlRo5tg7aZY', 'location_id': 'LETE3KSW0H1RB', 'payment_type': 'CARD_PRESENT', 'customer_id': '', 'payment_options': {'autocomplete': True, 'accept_partial_authorization': False}}}


In [17]:
class SearchOrderTool(BaseTool):
    name="search_order_tool"
    description="for searching for orders at a location"
    def _run(
          self, location_id: str, run_manager: Optional[CallbackManagerForToolRun] = None
      ) -> str:
          """Use the tool."""
          return client.orders.search_orders(body = {"location_ids": [location_id], "return_entries": True}).body['order_entries']

In [18]:
from langchain.tools import StructuredTool
from uuid import uuid4

from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class CreateOrderSchema(BaseModel):
    location_id: str = Field(description="Should be a valid location id provided in the prompt. ")
    name: str = Field(description="The name of the specified food item. If no food item is identified, choose a random one.")
    quantity: str = Field(description="**VERY IMPORTANT: must be a string. The quantity of food item requested.")
    amount: int = Field(description="**VERY IMPORTANT: absolutely must be an integer. The cost of the food item. ")
    currency: str = Field(description="The currency being paid with, defaults to 'USD'.")


class CreateOrderTool(BaseTool):
    name="create_new_order"
    description="used for creating a new order"
    args_schema: Type[CreateOrderSchema] = CreateOrderSchema
    def _run(
        self, location_id: str, name: str, quantity: str, amount: int, currency: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:    
          return client.orders.create_order(body = {  "order": { "location_id": location_id, "line_items": [{"name": name, "quantity": quantity, "modifiers": [], "base_price_money": { "amount": amount,"currency": currency}}]},"idempotency_key": str(uuid4().hex)})
          

In [19]:
from langchain.llms import vertexai
print(vertexai.VertexAI)
from vertexai.preview.language_models import TextGenerationModel
from langchain.chains import APIChain
from langchain.agents import AgentType, initialize_agent

tools = [CreateOrderTool(), SearchOrderTool()]

llm = vertexai.VertexAI(temperature=0)

agent = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

<class 'langchain.llms.vertexai.VertexAI'>


In [19]:
agent.run("location_id = LETE3KSW0H1RB, Hi my name is tristin, could I please order a cappaccino for 5 dollars?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "create_new_order",
  "action_input": {
    "location_id": "LETE3KSW0H1RB",
    "name": "Cappuccino",
    "quantity": "1",
    "amount": 500,
    "currency": "USD"
  }
}
```

Observation: <ApiResponse {"order": {"id": "iHLPW9xNW1h4SG8ubhqlRo5tg7aZY","location_id": "LETE3KSW0H1RB","line_items": [{"uid": "nAsmW4XGtk4I6V1H0dn5RD","quantity": "1","name": "Cappuccino","base_price_money": {"amount": 500,"currency": "USD"},"gross_sales_money": {"amount": 500,"currency": "USD"},"total_tax_money": {"amount": 0,"currency": "USD"},"total_discount_money": {"amount": 0,"currency": "USD"},"total_money": {"amount": 500,"currency": "USD"},"variation_total_price_money": {"amount": 500,"currency": "USD"},"item_type": "ITEM","total_service_charge_money": {"amount": 0,"currency": "USD"}}],"created_at": "2023-08-31T05:41:59.312Z","updated_at": "2023-08-31T05:41:59.312Z","state": "OPEN","version": 1,"total_tax_money": {"amount": 0,"currency"

'Your order for a cappuccino has been created. Your order ID is iHLPW9xNW1h4SG8ubhqlRo5tg7aZY.'